# Import Packages and Files

In [1]:
import dataloader as dl
import saveloader as sl

# Load Dataset

In [2]:
trainset, testset, classes = dl.load_cifar("./datasets")

Files already downloaded and verified
Files already downloaded and verified


In [3]:
trainloader, validloader, testloader = dl.make_dataloaders(trainset, testset, 5000, 128)

In [2]:
a = 1
b = 2
c = 3

sl.save_variable([a, b, c], "./variables/test.pkl")

IOError: [Errno 2] No such file or directory: './variables/test.pkl'

In [8]:
d, e, f = sl.load_variable("test.pkl")

In [9]:
print(d)
print(e)
print(f)

1
2
3
